# 데이터 불러오기

- 시드값

In [22]:
SEED = 42

- 데이터 불러오기

In [2]:
import pandas as pd
import numpy as np

DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Data/"

train_tr = pd.read_csv(f"{DATA_PATH}store_train_transactions.csv") # 학습용 구매기록 데이터
train_target = pd.read_csv(f"{DATA_PATH}store_train.csv") # 학습용 정답 데이터
test_tr = pd.read_csv(f"{DATA_PATH}store_test_transactions.csv") # 테스트용 구매기록 데이터
submit = pd.read_csv(f"{DATA_PATH}store_submission.csv") # 제출 양식 데이터

train_tr.shape , train_target.shape , test_tr.shape , submit.shape

((523105, 7), (14940, 2), (441196, 7), (12225, 2))

- 공통 피처 파일 불러오기

In [3]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Feature_csv/"

train_ft = pd.read_csv(f"{DATA_PATH}train_common_v2.0_1101.csv")
test_ft = pd.read_csv(f"{DATA_PATH}test_common_v2.0_1101.csv")

train_ft.shape , test_ft.shape

((14940, 465), (12225, 465))

# 결측치 처리

In [5]:
train_ft.isnull().sum().sum(), test_ft.isnull().sum().sum()

(0, 0)

# 특성 공학(Feature Engineering)

- ID 변수 제외

In [4]:
train_ft = train_ft.iloc[:,1:]
test_ft = test_ft.iloc[:,1:]
train_ft.shape, test_ft.shape

((14940, 464), (12225, 464))

- 추가 피처 만들어 보기

## Feature Encoding

In [5]:
cols = train_ft.select_dtypes("object").columns.tolist()
train_ft[cols].nunique()

주구매지점              4
주구매_중분류          246
주구매_대분류_수정         7
대분류_수정_평균금액최대      7
dtype: int64

In [6]:
import category_encoders as ce

In [7]:
enc = ce.one_hot.OneHotEncoder()
tmp = enc.fit_transform(train_ft[cols])
train_ft = pd.concat([train_ft,tmp],axis =1)

tmp = enc.transform(test_ft[cols])
test_ft = pd.concat([test_ft,tmp],axis =1)

train_ft.shape, test_ft.shape

((14940, 728), (12225, 728))

In [ ]:
# enc = ce.count.CountEncoder()
# train_ft["주구매_중분류_cnt"] = enc.fit_transform(train_ft[["주구매_중분류"]])
# test_ft["주구매_중분류_cnt"] = enc.transform(test_ft[["주구매_중분류"]])

# train_ft.shape, test_ft.shape

- 문자열 피처 삭제

In [8]:
cols

['주구매지점', '주구매_중분류', '주구매_대분류_수정', '대분류_수정_평균금액최대']

In [9]:
train_ft = train_ft.drop(columns=cols)
test_ft = test_ft.drop(columns=cols)
train_ft.shape, test_ft.shape

((14940, 724), (12225, 724))

In [10]:
train_ft.select_dtypes("object").columns , test_ft.select_dtypes("object").columns

(Index([], dtype='object'), Index([], dtype='object'))

## Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [12]:
train_ft[train_ft.columns] = scaler.fit_transform(train_ft)
test_ft[test_ft.columns] = scaler.transform(test_ft)
train_ft.head()

,내점일수,구매주기,주말방문비율,봄_구매비율,여름_구매비율,가을_구매비율,겨울_구매비율,주구매요일,일별평균구매건수,거래개월수,...,주구매_대분류_수정_5,주구매_대분류_수정_6,주구매_대분류_수정_7,대분류_수정_평균금액최대_1,대분류_수정_평균금액최대_2,대분류_수정_평균금액최대_3,대분류_수정_평균금액최대_4,대분류_수정_평균금액최대_5,대분류_수정_평균금액최대_6,대분류_수정_평균금액최대_7
0,-0.369867,0.002987,0.257728,-1.029777,0.001191,0.838272,0.338186,0.109631,-0.057297,0.017620,...,-0.154203,-0.378716,-0.187997,2.553208,-0.443004,-0.816155,-0.341822,-0.345596,-0.274521,-0.149122
1,0.144110,-0.356452,-1.008554,0.323951,-0.390607,0.620171,-0.552996,0.109631,0.222706,1.176010,...,-0.154203,-0.378716,-0.187997,-0.391664,2.257315,-0.816155,-0.341822,-0.345596,-0.274521,-0.149122
2,1.943028,-0.869935,0.036742,0.798943,-0.514333,-0.304527,-0.059266,-1.643370,0.277707,1.465608,...,-0.154203,-0.378716,-0.187997,-0.391664,2.257315,-0.816155,-0.341822,-0.345596,-0.274521,-0.149122
3,3.793345,-1.023980,-0.080558,0.420933,-0.327474,0.008592,-0.135636,0.109631,0.674668,1.465608,...,-0.154203,-0.378716,-0.187997,-0.391664,-0.443004,1.225257,-0.341822,-0.345596,-0.274521,-0.149122
4,0.452496,-0.613193,0.302875,-0.752532,1.707410,-0.130285,-0.821561,0.693965,0.679008,0.886413,...,-0.154203,-0.378716,-0.187997,-0.391664,2.257315,-0.816155,-0.341822,-0.345596,-0.274521,-0.149122


# 정답 데이터

In [13]:
target = train_target["target"]

# cv 점수 확인해보기

In [ ]:
# from lightgbm import LGBMClassifier

# model = LGBMClassifier(random_state=SEED)
# scores = cross_val_score(model,train_ft,target,cv = cv ,scoring='f1_macro',n_jobs = -1)
# np.mean(scores)

# 모델 학습

In [21]:
import numpy as np
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold

# 파라미터 설정
params = {
    "random_state": 42,
    "learning_rate": 0.1,  # 학습률
    "n_estimators": 500,   # 트리 개수
    "max_depth": 5,        # 최대 깊이
    "min_child_weight": 3, # 최소 자식 가중치
    "subsample": 0.8,      # 샘플링 비율
    "colsample_bytree": 0.8, # 특성 샘플링 비율
    "gamma": 0.1,           # 최소 손실 감소
    "n_jobs": -1,
    "early_stopping_rounds": 50,  
    "eval_metric": "logloss"
}

# F1 매크로 스코어와 모델을 저장할 리스트
scores = []
models = []

# Stratified K-Fold 교차 검증 설정
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 교차 검증 루프
for tri, vai in cv.split(train_ft, target):
    # 학습 데이터와 검증 데이터 분리
    x_train = train_ft.iloc[tri]
    y_train = target.iloc[tri]
    x_valid = train_ft.iloc[vai]
    y_valid = target.iloc[vai]

    # 모델 초기화 및 학습
    model = XGBClassifier(**params)
    model.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)   
    
    # 모델 저장
    models.append(model)

    # 예측 및 F1 매크로 스코어 계산
    pred = model.predict(x_valid)
    score = f1_score(y_valid, pred, average='macro')
    scores.append(score)

# F1 매크로 스코어의 평균 출력
print("Mean F1 Macro Score:", np.mean(scores))


Mean F1 Macro Score: 0.7166578765340972


In [25]:
import optuna
from sklearn.model_selection import StratifiedKFold, cross_val_score
import xgboost as xgb
from xgboost import XGBClassifier

# 목적함수를 클래스로 만들기
class Objective:
    # 변수 설정
    def __init__(self, x, y, seed):
        self.x = x
        self.y = y
        self.seed = seed
        self.cv = StratifiedKFold(5, shuffle= True, random_state= self.seed)

    def __call__(self, trial):  # 콜백함수 역할
        hp = {  "n_estimators": trial.suggest_int("n_estimators", 100, 1000, step=100),
                "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3, log=True),
                "max_depth": trial.suggest_int("max_depth", 3, 10),
                "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
                "subsample": trial.suggest_float("subsample", 0.5, 1.0),
                "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
                "gamma": trial.suggest_float("gamma", 0, 1)  }
        model = XGBClassifier(**hp, random_state= self.seed)    
        score = cross_val_score(model, self.x, self.y, cv= self.cv, scoring= "f1_macro", n_jobs= -1).mean()
        return score

# Sampler 객체 생성(대체모델 역할)
sampler = optuna.samplers.TPESampler(seed= SEED)

# study 객체 생성
study = optuna.create_study(direction= "maximize", sampler= sampler)

# optimize 매서드 실행
objective_func = Objective(train_ft, target, SEED)
study.optimize(objective_func, n_trials= 200) 

print("Best trial:")
print(study.best_trial.params)

[I 2024-11-01 19:52:04,432] A new study created in memory with name: no-name-9fd4df1a-45d5-493a-bf3b-3b4400081050
[I 2024-11-01 19:52:40,836] Trial 0 finished with value: 0.6912295212558891 and parameters: {'n_estimators': 400, 'learning_rate': 0.2536999076681772, 'max_depth': 8, 'min_child_weight': 6, 'subsample': 0.5780093202212182, 'colsample_bytree': 0.5779972601681014, 'gamma': 0.05808361216819946}. Best is trial 0 with value: 0.6912295212558891.
[I 2024-11-01 19:53:49,906] Trial 1 finished with value: 0.710595762870085 and parameters: {'n_estimators': 900, 'learning_rate': 0.07725378389307355, 'max_depth': 8, 'min_child_weight': 1, 'subsample': 0.9849549260809971, 'colsample_bytree': 0.9162213204002109, 'gamma': 0.21233911067827616}. Best is trial 1 with value: 0.710595762870085.
[I 2024-11-01 19:54:06,875] Trial 2 finished with value: 0.6991820134892603 and parameters: {'n_estimators': 200, 'learning_rate': 0.018659959624904916, 'max_depth': 5, 'min_child_weight': 6, 'subsample'

Best trial:
{'n_estimators': 800, 'learning_rate': 0.04036413044768581, 'max_depth': 4, 'min_child_weight': 7, 'subsample': 0.7505214930635562, 'colsample_bytree': 0.6290102054237857, 'gamma': 0.648553153047272}


In [36]:
study.best_params

{'n_estimators': 800,
 'learning_rate': 0.04036413044768581,
 'max_depth': 4,
 'min_child_weight': 7,
 'subsample': 0.7505214930635562,
 'colsample_bytree': 0.6290102054237857,
 'gamma': 0.648553153047272}

In [31]:
params = study.best_params

model = XGBClassifier(**params)
model.fit(train_ft, target)
pred = model.predict(test_ft)

In [34]:
from flaml import AutoML
import xgboost 
from xgboost import XGBRFClassifier

auto_ml = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*60,
           "seed" : 42,
           "early_stop" : True,
           "estimator_list" : ['xgboost']  }   
auto_ml.fit(train_ft, target, **params)

[flaml.automl.logger: 11-01 22:14:12] {1728} INFO - task = classification
[flaml.automl.logger: 11-01 22:14:12] {1739} INFO - Evaluation method: holdout
[flaml.automl.logger: 11-01 22:14:12] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-01 22:14:12] {1955} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl.logger: 11-01 22:14:12] {2258} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 11-01 22:14:13] {2393} INFO - Estimated sufficient time budget=5392s. Estimated necessary time budget=5s.
[flaml.automl.logger: 11-01 22:14:13] {2442} INFO -  at 2.3s,	estimator xgboost's best error=0.6223,	best estimator xgboost's best error=0.6223
[flaml.automl.logger: 11-01 22:14:13] {2258} INFO - iteration 1, current learner xgboost
[flaml.automl.logger: 11-01 22:14:13] {2442} INFO -  at 2.9s,	estimator xgboost's best error=0.6223,	best estimator xgboost's best error=0.6223
[flaml.automl.logger: 11-01 22:14:13] {2258} INFO - iteration 2, c

In [47]:
1- auto_ml.best_loss

0.7190496981014807

In [35]:
auto_ml.best_config

{'n_estimators': 1150,
 'max_leaves': 47,
 'min_child_weight': 0.7910763321600376,
 'learning_rate': 0.056604412387466024,
 'subsample': 0.8235981634894725,
 'colsample_bylevel': 1.0,
 'colsample_bytree': 0.8447620742715861,
 'reg_alpha': 0.0011926146429896113,
 'reg_lambda': 2.9660853748175002}

In [37]:
pred = auto_ml.predict(test_ft)

In [50]:
from flaml import AutoML
import xgboost 
from xgboost import XGBRFClassifier

auto_ml = AutoML()
params = {
    "metric": "macro_f1",                # 평가 지표
    "task": "classification",            # 분류 작업
    "time_budget": 2 * 60 * 60,          # 시간 제한 (2시간)
    "seed": 42,                          # 랜덤 시드 설정
    "early_stop": True,                  # 조기 종료 활성화
    "estimator_list": ["xgboost"],  # 사용 모델 리스트
    "eval_method": "cv",                 # 교차 검증
    "n_splits": 10,                      # Stratified K-Fold로 10분할 교차 검증
    "log_type": "plain_text"             # 로그 출력 형식 설정
}  
auto_ml.fit(train_ft, target, **params)

print("Best model:", auto_ml.best_estimator)
print("Best hyperparameters:", auto_ml.best_config)
print("Best macro F1 score:", auto_ml.best_loss)

[flaml.automl.logger: 11-01 23:29:09] {1728} INFO - task = classification
[flaml.automl.logger: 11-01 23:29:09] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 11-01 23:29:09] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-01 23:29:09] {1955} INFO - List of ML learners in AutoML Run: ['xgboost']
[flaml.automl.logger: 11-01 23:29:09] {2258} INFO - iteration 0, current learner xgboost
[flaml.automl.logger: 11-01 23:29:16] {2393} INFO - Estimated sufficient time budget=67793s. Estimated necessary time budget=68s.
[flaml.automl.logger: 11-01 23:29:16] {2442} INFO -  at 8.2s,	estimator xgboost's best error=0.6223,	best estimator xgboost's best error=0.6223
[flaml.automl.logger: 11-01 23:29:16] {2258} INFO - iteration 1, current learner xgboost
[flaml.automl.logger: 11-01 23:29:23] {2442} INFO -  at 15.4s,	estimator xgboost's best error=0.6223,	best estimator xgboost's best error=0.6223
[flaml.automl.logger: 11-01 23:29:23] {2258} INFO - iteration 2, cur

In [51]:
1-auto_ml.best_loss

0.7183938846393594

In [52]:
pred = auto_ml.predict(test_ft)

In [14]:
from flaml import AutoML
import xgboost 
from xgboost import XGBClassifier

auto_ml_ens = AutoML()
params = { "metric" : "macro_f1",
           "task" : "classification",
           "time_budget" : 60*60*5,
           "seed" : 42,
           "early_stop" : True,
           "eval_method": "cv",                 # 교차 검증
           "n_splits": 10,                      # Stratified K-Fold로 10분할 교차 검증
           "ensemble" : {'final_estimator' : XGBClassifier() },    # 메타모델이 로지스틱 회귀!
           "estimator_list" : ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2', 'kneighbor']  }   # 앙상블에 사용할 모델 지정

auto_ml_ens.fit(train_ft, target, **params)

[flaml.automl.logger: 11-02 01:29:22] {1728} INFO - task = classification
[flaml.automl.logger: 11-02 01:29:22] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 11-02 01:29:22] {1838} INFO - Minimizing error metric: 1-macro_f1
[flaml.automl.logger: 11-02 01:29:22] {1955} INFO - List of ML learners in AutoML Run: ['catboost', 'lgbm', 'rf', 'xgboost', 'histgb', 'lrl1', 'lrl2', 'kneighbor']
[flaml.automl.logger: 11-02 01:29:22] {2258} INFO - iteration 0, current learner catboost
[flaml.automl.logger: 11-02 01:30:00] {2393} INFO - Estimated sufficient time budget=377128s. Estimated necessary time budget=592s.
[flaml.automl.logger: 11-02 01:30:00] {2442} INFO -  at 39.2s,	estimator catboost's best error=0.2928,	best estimator catboost's best error=0.2928
[flaml.automl.logger: 11-02 01:30:00] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 11-02 01:30:07] {2442} INFO -  at 46.8s,	estimator lgbm's best error=0.6223,	best estimator catboost's best error=0.2928
[f

In [15]:
pred = auto_ml_ens.predict(test_ft)

In [26]:
1 - auto_ml_ens.best_loss

0.721442073758231

In [19]:
auto_ml_ens.best_config

{'n_estimators': 1852,
 'num_leaves': 4,
 'min_child_samples': 101,
 'learning_rate': 0.10307534052139548,
 'log_max_bin': 5,
 'colsample_bytree': 0.7777111765274982,
 'reg_alpha': 0.09452200380404109,
 'reg_lambda': 17.38285461630384}

In [20]:
auto_ml_ens.best_estimator

'lgbm'

In [23]:
from lightgbm import LGBMClassifier
params = auto_ml_ens.best_config
model = LGBMClassifier(**params, random_state= 42)
model.fit(train_ft, target)
pred = model.predict(test_ft)

# 평가를 위한 제출 파일 생성
- 예측 결과를 target 컬럼에 넣어 csv 파일로 저장후에 제출한다.

In [24]:
submit["target"] = pred
submit

,ID,target
0,test_0,0.0
1,test_1,0.0
2,test_2,0.0
3,test_3,1.0
4,test_4,0.0
...,...,...
12220,test_12220,1.0
12221,test_12221,0.0
12222,test_12222,0.0
12223,test_12223,0.0


- 예측 결과를 csv 파일로 저장하여 제출

In [25]:
DATA_PATH = "C:/Users/user/Desktop/데이터분석/05 Project_Final/Submit_csv/"
submit.to_csv(f"{DATA_PATH}submit_v2.0_lgbm.csv",index=False)